### Notebook to parse scraped html to produce cleaned text of FC decisions

Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International [(CC BY-NC 4.0)](https://creativecommons.org/licenses/by-nc/4.0/). NOTE: Users must also comply with upstream [licensing](https://www.fct-cf.gc.ca/en/pages/important-notices) for the FC data source, as well as requests on source urls not to allow indexing of the documents by search engines to protect privacy. As a result, users must not make the data available in formats or locations that can be indexed by search engines.

Dataset & Code to be cited as: 

    Sean Rehaag, "Federal Court Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/fc/>.

Notes:

(1) Data Source: [Federal Court](https://www.fct-cf.gc.ca). 

(2) Unofficial Data: The data are unofficial reproductions of materials on the Federal Court website. Links to official versions are included in the dataset.

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Federal Court.

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, see the Federal Court of Appeal website's [Terms of Use](https://www.fct-cf.gc.ca/en/pages/important-notices).

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information. 

(6) Limitation: Only includes cases with neutral citation, which began to be used in 2001

(7) Delay: Decisions may take many months to be translated (sometimes over a year). As a result, in the most recent years, decisions may only be available in one language.

### Requirements:

    pip install pandas
    pip install requests

(Written on Python 3.9.12)




In [1]:
# import libraries
from bs4 import BeautifulSoup
import pandas as pd
import re
import pathlib
import json
import random

# set up progress bar
from tqdm import tqdm
tqdm.pandas()

# set paths
in_path = pathlib.Path('d:/scraping/DATA/DECISIONS/FC/BULK/HTML/')
out_path_raw = pathlib.Path('DATA/fc_raw.jsonl')
out_path_parsed = pathlib.Path('DATA/fc_cases.jsonl')
out_path_parquet = pathlib.Path('DATA/fc_cases.parquet')
out_path_yearly = pathlib.Path('DATA/YEARLY/')
out_path_HF = pathlib.Path('d:/AI-Projects/canadian-legal-data/FC/train.parquet')

# set years sought
start_year = 2001 
end_year = 2024


### Load Raw Data

In [2]:
# get list of files (including subdirectories) using pathlib
files = list(in_path.glob('**/*.json'))
print(len(files))

# Load data from files
results = []
for file in tqdm(files):
    with open(file) as f:
        data = json.load(f)
        results.append(data)

# convert list of dictionaries to dataframe
df = pd.DataFrame(results)




75212


100%|██████████| 75212/75212 [07:25<00:00, 168.88it/s]


In [3]:
# export raw df to jsonl
df.to_json(out_path_raw, orient='records', lines=True)


### Parse Data

In [4]:
# clean dataframe

#remove ?iframe=true from scraped_link
df['scraped_link'] = df['scraped_link'].str.replace('\?iframe=true', '', regex=True)

# remove T and everything after from scraped_timestamp & case_decision_date
df['scraped_timestamp'] = df['scraped_timestamp'].str.replace('T.*', '', regex=True)
df['case_decision_date'] = df['case_decision_date'].str.replace('T.*', '', regex=True)

# remove scraped_status_code, referrer_main_source, referrer_sub_source, referrer_file, referrer_timestamp
df = df.drop(columns=['scraped_status_code', 'referrer_main_source', 'referrer_sub_source', 'referrer_file', 'referrer_timestamp'])

# convert case_year to int and filter for years sought
df['case_year'] = df['case_year'].astype(int)
df = df[df.case_year >= start_year]
df = df[df.case_year <= end_year]

# remove cases where no citation (typically orders or errors)
# remove df where citation includes '=' or '-'
df = df[df.case_citation.str.contains('=') == False]
df = df[df.case_citation.str.contains('-') == False]
# remove df language is 'en' and case_citation does not include 'FC'
df = df[~((df.case_language == 'en') & (df.case_citation.str.contains('FC') == False))]
# remove df language is 'fr' and case_citation does not include 'CF'
df = df[~((df.case_language == 'fr') & (df.case_citation.str.contains('CF') == False))]

# if citation2 = citation 1, make citation2 = ''
df['case_citation2'] = df['case_citation2'].where(df['case_citation2'] != df['case_citation'], '')

# fill nan in citation2 with ''
df['case_citation2'] = df['case_citation2'].fillna('')

# change 'scraped_link' column to 'source_url;
df = df.rename(columns={'scraped_link': 'source_url'})

# remove 'case_' from all column names
df.columns = df.columns.str.replace('case_', '')


In [5]:
# Run this in parallel to speed up

import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

# Extract text of cases from html
def get_text(html):

    # if html is None, return None
    if html is None:
        return None
    
    # extract text from class 'entry-content' in page1 html using beautiful soup
    soup = BeautifulSoup(html, 'html.parser')

    # convert <br> to new line to preserve paragraphs
    for br in soup.find_all('br'):
        br.replace_with('\n')

    # Insert newline characters after each <p> tag to preserve paragraphs
    for p in soup.find_all('p'):
        p.insert_after('\n')

    return soup.text

# Convert the pandas DataFrame to a Dask DataFrame
ddf = dd.from_pandas(df, npartitions=16) 

# Apply the function in parallel
ddf['text'] = ddf.html.apply(get_text)

# Convert the Dask DataFrame back to a pandas DataFrame
with dask.config.set(scheduler='processes'):
    with ProgressBar():
        df = ddf.compute()




[                                        ] | 0% Completed | 253.70 us

d:\AI-Projects\venv311\Lib\site-packages\dask\dataframe\core.py:4378: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('html', 'object'))

  warnings.warn(meta_warning(meta))


[########################################] | 100% Completed | 398.31 s


In [6]:
# Clean text of cases
def clean_text(text):

    # remove \xa0
    text = text.replace('\xa0', ' ')

    # Remove multiple whitespaces and preserve paragraphs
    text = '\n'.join([re.sub(r'\s+', ' ', line.strip()) for line in text.split('\n')])
    
    # Remove single newlines
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # Convert multiple newlines to single newlines
    text = re.sub(r'\n+', '\n', text)

    # Remove 'You are being directed to the most recent version...
    if 'You are being directed to the most recent version of the statute which may not be' in text:
        text = text.split('You are being directed to the most recent version of the statute which may not be')[0]

    # Remove 'Vous allez être redirigé vers la version...'
    if '\nVous allez être redirigé vers la version' in text:
        text = text.split('\nVous allez être redirigé vers la version')[0]

    # if '\nDecision Information\n' in text, remove everything before it
    if '\nDecision Information\n' in text:
        text = text.split('\nDecision Information\n')[1]

    # if '\nInformations sur la décision\n' in text, remove everything before it
    if '\nInformations sur la décision\n' in text:
        text = text.split('\nInformations sur la décision\n')[1]

    # Remove all strings '\n[Page #]\n' (with # being a number of up to 4 digits 
    text = re.sub(r'\n\[Page \d{1,3}\]\n', ' ', text)
    
    return text

df['unofficial_text'] = df.text.progress_apply(clean_text)


100%|██████████| 62705/62705 [01:24<00:00, 745.24it/s] 


In [7]:
# drop unneeded columns
df = df.drop(columns=['html'])
df = df.drop(columns=['text'])

# rename decision_date to document_date
df = df.rename(columns={'decision_date': 'document_date'})

# add dataset column
df['dataset'] = 'FC'

# add other column
df['other'] = ''

# reorder columns
list_cols = ['citation',
             'citation2', 
             'dataset', 
             'year', 
             'name', 
             'language', 
             'document_date', 
             'source_url', 
             'scraped_timestamp', 
             'unofficial_text',
             'other']

df = df[list_cols]

# reset index
df = df.reset_index(drop=True)


In [8]:
df.head()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
0,2001 FCT 1,,FC,2001,Adecon Ship Management Inc. v. Cuba,en,2001-02-01,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-18,Adecon Ship Management Inc. v. Cuba\nCourt (s)...,
1,2001 FCT 10,,FC,2001,Islam v. Canada (Minister of Citizenship and I...,en,2001-02-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-18,Islam v. Canada (Minister of Citizenship and I...,
2,2001 FCT 100,,FC,2001,Duterville v. Canada,en,2001-02-20,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-18,Duterville v. Canada\nCourt (s) Database\nFede...,
3,2001 FCT 1000,,FC,2001,LS Entertainment Group Inc. v. KALOS VISION LT...,en,2001-09-07,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-18,LS Entertainment Group Inc. v. KALOS VISION LT...,
4,2001 FCT 1001,,FC,2001,Ay v. Canada (Minister of Citizenship and Immi...,en,2001-09-07,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-18,Ay v. Canada (Minister of Citizenship and Immi...,


In [9]:
df.tail()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
62700,2024 CF 981,,FC,2024,Bekkai c. Canada (Procureur général),fr,2024-06-25,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-09-08,Bekkai c. Canada (Procureur général)\nBase de ...,
62701,2024 CF 982,,FC,2024,Peralta Rojas c. Canada (Citoyenneté et Immigr...,fr,2024-07-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Peralta Rojas c. Canada (Citoyenneté et Immigr...,
62702,2024 CF 983,,FC,2024,Butt et al v. Canada,fr,2024-06-25,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-09-07,Butt c. Canada\nBase de données – Cour (s)\nDé...,
62703,2024 CF 984,,FC,2024,Sanchez Herrera c. Canada (Sécurité Publique e...,fr,2024-06-25,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Sanchez Herrera c. Canada (Sécurité publique e...,
62704,2024 CF 995,,FC,2024,Lemay Co Inc. c. Canada (le Procureur Général),fr,2024-06-26,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2024-07-12,Lemay Co Inc. c. Canada (Procureur général)\nB...,


In [10]:
# count number of cases per year
df.groupby('year').count()

,citation,citation2,dataset,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
year,,,,,,,,,,
2001,2806,2806,2806,2806,2806,2806,2806,2806,2806,2806
2002,2638,2638,2638,2638,2638,2638,2638,2638,2638,2638
2003,2961,2961,2961,2961,2961,2961,2961,2961,2961,2961
2004,3506,3506,3506,3506,3506,3506,3506,3506,3506,3506
2005,3408,3408,3408,3408,3408,3408,3408,3408,3408,3408
2006,3069,3069,3069,3069,3069,3069,3069,3069,3069,3069
2007,2735,2735,2735,2735,2735,2735,2735,2735,2735,2735
2008,2797,2797,2797,2797,2797,2797,2797,2797,2797,2797
2009,2499,2499,2499,2499,2499,2499,2499,2499,2499,2499


### Export data

In [11]:
# export cleaned df to jsonl
df.to_json(out_path_parsed, orient='records', lines=True)

In [12]:
# export cleaned df to parquet
df.to_parquet(out_path_parquet)

In [13]:
# export cleaned df to yearly json files
for year in tqdm(range(start_year, end_year+1)):
    for language in ['en', 'fr']:
        df[(df.year == year) & (df.language == language)].to_json(out_path_yearly / f'{year}_{language}.json', orient='records', indent=4)
    

100%|██████████| 24/24 [00:04<00:00,  4.89it/s]


In [14]:
# export to HF format
df.to_parquet(out_path_HF)